In [ ]:
import sys
import qgrid
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymatgen import Structure

sys.path.append('../../lib/')
from dftmanlib.pwscf import (pwinput_helper,
                             pwcalculation_helper,
                             pseudo_helper)
from dftmanlib.pwscf.workflow import EOSWorkflow
from dftmanlib.job import SubmitJob, submitjob_statuses, submit_status
from dftmanlib.matproj import mpquery_helper

# PSEUDO_TABLE = '/data/tools/shared/dftman/pseudo_table.json'
PSEUDO_TABLE = '/home/azadoks/.local/share/pseudo/pseudo_table.json'
# available pseudo families:
# ['SSSP_EFFICIENCY', 'SSSP_PRECISION', 'GBRV_US_LDA',
#  'GBRV_US_PBE', 'GBRV_US_PBEsol', 'DOJO_STANDARD_LDA_NC',
#  'DOJO_STANDARD_PBE_NC', 'DOJO_STANDARD_PBEsol_NC',
#  'DOJO_STRINGENT_LDA_NC', 'DOJO_STRINGENT_PBE_NC',
#  'DOJO_STRINGENT_PBAsol_NC']
PSEUDO_FAMILY = 'GBRV_US_PBE'

qgrid.enable()
MP_API_KEY = '0WqdPfXxloze6T9N'
# db = init()

In [1]:
from tinydb import TinyDB
db = TinyDB('test.json')

In [5]:
db.storage_proxy_class

<function StorageProxy.mro>

## Materials Project Query

In [ ]:
criteria = {
    'elements': 'Al',
    'nsites': 1,
    'spacegroup.number': 225,
}
properties = []
m = mpquery_helper(criteria, properties, MP_API_KEY)
m.query()
m.display()

### Base configuration

In [ ]:
structure = Structure.from_dict(m.result[0]['structure'])
pseudo = pseudo_helper(structure, PSEUDO_FAMILY,
                       PSEUDO_TABLE)
base_inputs = {
        'structure': structure,

        'control': {
            'calculation': 'relax',
            'verbosity': 'high',
            'disk_io': 'none',
        },
        'system': {
            'ibrav': 0,
            'ecutwfc': 45,
            'occupations': 'smearing',
            'degauss': 0.01,
            'smearing': 'mv',
        },
        'electrons': {
            'electron_maxstep': 500,
            'conv_thr': 1.0e-6,
        },
        'ions': {},
        'cell': {},
        'kpoints_mode': 'automatic',
        'kpoints_grid': (20, 20, 20),
        'kpoints_shift': (0, 0, 0),

        'pseudo': pseudo
    }

In [ ]:
calculation = pwcalculation_helper(**base_inputs,
                                   additional_inputs = list(pseudo.values()))

In [ ]:
calc_json = json.dumps(calculation, cls=MontyEncoder)

In [ ]:
loaded_calc = json.loads(calc_json, cls=MontyDecoder)

In [ ]:
loaded_calc

In [ ]:
runname = 'test1'

job = SubmitJob(calculation, code='espresso-6.2.1_pw', runname=runname)

key = store(job, db)
job.submit()

In [ ]:
job_key = '981986e82a27'
job = db.Jobs[job_key]

In [ ]:
job.check_status()

In [ ]:
# workflow = EOSWorkflow(structure, pseudo, base_inputs, db)
# # workflow.run()

In [ ]:
# display(workflow.check_status().set_index('ID')[['Run Name', 'Status', 'Location', 'Submission Time', 'Key']])
# display(submit_status())